In [1]:
import sys
!{sys.executable} -m pip install pulp 

import pulp
# Import PuLP modeler functions
from pulp import * 

import numpy
import matplotlib as plt
import pandas as pd

# Seaborn for pretty graphs
import seaborn

# May do plotly instead for interactivity

## Context: Marine Spatial Planning and a Linear Programming Approach

## The Decision Variables

## The Constraints

## Accounting for minute changes in our variables - Sensitivity Analysis